In [373]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Flatten , Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model #모델 시각화 
from tensorflow.keras.optimizers import Adam


tf.config.run_functions_eagerly(False)



## 데이터 불러오기
- 앞서 크롤링하여 엑셀에 저장한 파일을 불러온다.

In [374]:
data_df = pd.read_excel('baseball_data.xlsx')
target_df = pd.read_excel('label.xlsx')




## 훈련세트, 검증세트, 테스트세트 분리
- 8:2비율로 훈련세트, 테스트 세트를 분리하고 또 다시 8:2로 훈련세트, 검증세트를 분리한다.
- 스코어를 예측하는 회귀모델을 위해 훈련세트, 검증세트, 테스트세트를 A팀과 B팀으로 분리한다.
- 레이블도 A팀 스코어 레이블, B팀 스코어 레이블, 승패 레이블로 분리한다.

In [375]:
#훈련세트, 검증세트, 테스트세트 분리 
X_full_set, X_test=  train_test_split(data_df, test_size = 0.2, random_state = 7)
y_full_set, y_test = train_test_split(target_df, test_size = 0.2, random_state = 7)
x_train, x_val, y_train, y_val = train_test_split(X_full_set,y_full_set, test_size=0.2, random_state=7)

#A B 나누기
A_train = x_train.iloc[:,:14]
B_train = x_train.iloc[:,14:]
A_valid = x_val.iloc[:,:14]   
B_valid = x_val.iloc[:,14:]
A_test = X_test.iloc[:,:14]
B_test = X_test.iloc[:,14:]

winlose_target_train = pd.DataFrame(y_train['승패'])
A_score_train = pd.DataFrame(y_train['A팀_스코어'])
B_score_train = pd.DataFrame(y_train['B팀_스코어'])

winlose_target_valid = pd.DataFrame(y_val['승패'])
A_score_valid = pd.DataFrame(y_val['A팀_스코어'])
B_score_valid = pd.DataFrame(y_val['B팀_스코어'])

winlose_target_test = pd.DataFrame(y_test['승패'])
A_score_test = pd.DataFrame(y_test['A팀_스코어'])
B_score_test = pd.DataFrame(y_test['B팀_스코어'])
display(X_test)


,A_타석,A_타수,A_안타,A_홈런,A_볼넷,A_사구,A_삼진,A_땅볼,A_플라이,A_팀타율,...,B_볼넷,B_사구,B_삼진,B_땅볼,B_플라이,B_팀타율,B_출루율,B_잔루,B_상대선발투수ERA,B_상대선발투수WHIP
531,41,32,8,0,5,3,11,6,7,0.250000,...,0,0,8,9,7,0.225806,0.184211,9,5.29,1.70
1555,58,51,27,3,5,1,5,10,9,0.529412,...,7,0,9,12,4,0.242424,0.294118,21,2.31,1.30
140,35,31,6,2,4,0,5,9,11,0.193548,...,1,0,7,13,7,0.129032,0.125000,6,3.50,1.26
1516,34,34,9,0,0,0,7,6,12,0.264706,...,1,0,6,9,10,0.294118,0.234043,17,5.71,1.44
1378,32,30,5,0,0,1,8,13,4,0.166667,...,3,3,7,9,9,0.342105,0.322034,18,6.04,1.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,46,32,5,1,14,0,7,6,14,0.156250,...,6,0,11,8,7,0.161290,0.215686,18,1.66,1.12
946,37,34,8,0,2,1,4,7,15,0.235294,...,4,0,9,6,8,0.378378,0.339623,20,3.22,1.27
1597,35,33,6,1,2,0,9,6,12,0.181818,...,2,0,9,4,10,0.323529,0.270833,16,2.61,1.09
975,44,36,13,0,6,0,5,8,10,0.361111,...,3,0,11,10,4,0.242424,0.250000,16,5.58,1.46


## 데이터 전처리
- min-max 스케일링(정규화) 와 표준화 둘 중 더 좋은걸로 선택

In [376]:
#데이터 정규화
scaler = StandardScaler() 
mms = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_val=scaler.fit_transform(x_val)
X_test=scaler.transform(X_test)

In [377]:
A_train= scaler.fit_transform(A_train)
B_train= scaler.fit_transform(B_train)

A_valid=scaler.fit_transform(A_valid)
B_valid=scaler.fit_transform(B_valid)

A_test=scaler.transform(A_test)
B_test=scaler.transform(B_test)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- A_땅볼
- A_볼넷
- A_사구
- A_삼진
- A_상대선발투수ERA
- ...
Feature names seen at fit time, yet now missing:
- B_땅볼
- B_볼넷
- B_사구
- B_삼진
- B_상대선발투수ERA
- ...

  warnings.warn(message, FutureWarning)


In [378]:
#차원축소
#pca = PCA(n_components=7)
#X_train = pca.fit_transform(X_train)
#X_valid = pca.fit_transform(X_valid)


## 모델 만들기

In [379]:
#모델1 만들기
input_A = Input(shape=(14,),name='input_A')
input_B = Input(shape=(14,),name='input_B')
hidden_A_1 = Dense(16,activation='relu',name='hidden_A_1')(input_A)
hidden_A_2 = Dense(8,activation='relu',name='hidden_A_2')(hidden_A_1)
hidden_A_3= Dense(4,activation='relu',name='hidden_A_3')(hidden_A_2)
hidden_B_1 = Dense(16,activation='relu',name='hidden_B_1')(input_B)
hidden_B_2 = Dense(8,activation='relu',name='hidden_B_2')(hidden_B_1)
hidden_B_3 = Dense(4,activation='relu',name='hidden_B_3')(hidden_B_2)
output_A = Dense(1,name='output_A')(hidden_A_3)
output_B = Dense(1,name='output_B')(hidden_B_3)

model_1 = Model(inputs=[input_A,input_B] , outputs=[output_A,output_B])

#모델2 만들기
input= Input(shape=(28,),name='input')
hidden_1 = Dense(64,activation='relu',name='hidden_1')(input)
hidden_2 = Dense(32,activation='relu',name='hidden_2')(hidden_1)
hidden_3 = Dense(16,activation='relu',name='hidden_3')(hidden_2)
hidden_4 = Dense(8,activation='relu',name='hidden_4')(hidden_3)
output = Dense(1,activation='sigmoid',name='output')(hidden_4)

model_2 = Model(inputs=input,outputs=output)

## 모델컴파일
- 회귀문제 : MSE(loss) , MAE(eval)

In [380]:
model_1.compile(loss='mse',
              optimizer=Adam(learning_rate=1e-2),metrics=['mae'])

model_2.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['accuracy'])

In [381]:
history = model_1.fit([A_train,B_train],
                    [A_score_train,B_score_train],
                    epochs=50, validation_data=([A_valid,B_valid],[A_score_valid,B_score_valid]))

Epoch 1/50
33/33 [==============================] - 2s 12ms/step - loss: 34.7117 - output_A_loss: 11.7512 - output_B_loss: 22.9605 - output_A_mae: 2.3749 - output_B_mae: 3.5884 - val_loss: 9.7749 - val_output_A_loss: 3.5964 - val_output_B_loss: 6.1786 - val_output_A_mae: 1.4873 - val_output_B_mae: 1.8350
Epoch 2/50
33/33 [==============================] - 0s 5ms/step - loss: 7.1967 - output_A_loss: 3.4030 - output_B_loss: 3.7937 - output_A_mae: 1.4093 - output_B_mae: 1.5169 - val_loss: 6.2797 - val_output_A_loss: 2.8012 - val_output_B_loss: 3.4785 - val_output_A_mae: 1.3082 - val_output_B_mae: 1.4507
Epoch 3/50
33/33 [==============================] - 0s 3ms/step - loss: 5.4716 - output_A_loss: 2.5860 - output_B_loss: 2.8855 - output_A_mae: 1.2628 - output_B_mae: 1.3295 - val_loss: 5.4867 - val_output_A_loss: 2.4399 - val_output_B_loss: 3.0467 - val_output_A_mae: 1.1986 - val_output_B_mae: 1.3590
Epoch 4/50
33/33 [==============================] - 0s 4ms/step - loss: 4.9318 - output_A_

33/33 [==============================] - 0s 4ms/step - loss: 3.8450 - output_A_loss: 1.6853 - output_B_loss: 2.1597 - output_A_mae: 1.0006 - output_B_mae: 1.1344 - val_loss: 4.8388 - val_output_A_loss: 2.0665 - val_output_B_loss: 2.7723 - val_output_A_mae: 1.0926 - val_output_B_mae: 1.2942
Epoch 29/50
33/33 [==============================] - 0s 3ms/step - loss: 3.8538 - output_A_loss: 1.6817 - output_B_loss: 2.1721 - output_A_mae: 0.9989 - output_B_mae: 1.1375 - val_loss: 4.5355 - val_output_A_loss: 1.9995 - val_output_B_loss: 2.5360 - val_output_A_mae: 1.0757 - val_output_B_mae: 1.2466
Epoch 30/50
33/33 [==============================] - 0s 3ms/step - loss: 3.8891 - output_A_loss: 1.7591 - output_B_loss: 2.1301 - output_A_mae: 1.0265 - output_B_mae: 1.1380 - val_loss: 5.3451 - val_output_A_loss: 2.3869 - val_output_B_loss: 2.9582 - val_output_A_mae: 1.1575 - val_output_B_mae: 1.3380
Epoch 31/50
33/33 [==============================] - 0s 3ms/step - loss: 3.8771 - output_A_loss: 1.7772

In [382]:
history = model_2.fit(x_train,winlose_target_train, epochs=50, validation_data=(x_val,winlose_target_valid))

Epoch 1/50
33/33 [==============================] - 1s 8ms/step - loss: 0.6062 - accuracy: 0.7683 - val_loss: 0.5191 - val_accuracy: 0.8301
Epoch 2/50
33/33 [==============================] - 0s 3ms/step - loss: 0.4042 - accuracy: 0.8726 - val_loss: 0.3686 - val_accuracy: 0.8571
Epoch 3/50
33/33 [==============================] - 0s 2ms/step - loss: 0.2795 - accuracy: 0.8880 - val_loss: 0.3217 - val_accuracy: 0.8687
Epoch 4/50
33/33 [==============================] - 0s 3ms/step - loss: 0.2407 - accuracy: 0.9035 - val_loss: 0.3071 - val_accuracy: 0.8571
Epoch 5/50
33/33 [==============================] - 0s 3ms/step - loss: 0.2213 - accuracy: 0.9083 - val_loss: 0.3059 - val_accuracy: 0.8494
Epoch 6/50
33/33 [==============================] - 0s 3ms/step - loss: 0.2065 - accuracy: 0.9199 - val_loss: 0.3152 - val_accuracy: 0.8533
Epoch 7/50
33/33 [==============================] - 0s 3ms/step - loss: 0.1952 - accuracy: 0.9199 - val_loss: 0.3156 - val_accuracy: 0.8533
Epoch 8/50
33/33 [==

## 모델 평가

In [384]:
model_1.evaluate([A_test,B_test],[A_score_test,B_score_test])
model_2.evaluate(X_test,winlose_target_test)

11/11 [==============================] - 0s 1ms/step - loss: 0.5053 - accuracy: 0.8765


[0.5052766799926758, 0.8765432238578796]

## 예측

In [ ]:
model_1.predict()
model_2.predict()